<a href="https://colab.research.google.com/github/gargs13/24B1837-Build-your-own-GPT-bot/blob/main/sent_analysis_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from transformers import BertTokenizer as bt
from transformers import BertModel as bm
from transformers import BertForSequenceClassification as bc
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
import torch.nn as nn
from collections import Counter
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import torch

In [66]:
df = pd.read_csv('//content//IMDB Dataset.csv')
#df = df.sample(10000, random_state=42).reset_index(drop=True)
reviews=[]
sentiments=[]
for i in range(len(df['review'])):
    reviews.append(df["review"][i])

sentiments = df['sentiment'].map({'positive': 1, 'negative': 0})

In [19]:
sentiments[0]

np.int64(1)

In [67]:
class ReviewDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.lengths = [len(word_tokenize(review)) for review in data]
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx], self.lengths[idx]


In [68]:
bt=bt.from_pretrained('bert-base-uncased')

def format(batch):
  texts, labels, lengths = zip(*batch)
  tokens = bt(list(texts), padding=True, truncation=True, max_length=128, return_tensors="pt")
  labels = torch.tensor(labels, dtype=torch.long)
  return tokens, labels

In [69]:
dataset = ReviewDataset(reviews, sentiments)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=format)
val_dl = DataLoader(val_ds, batch_size=32, collate_fn=format)


In [70]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = bc.from_pretrained('bert-base-uncased', num_labels=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5, weight_decay=1e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [71]:
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for tokens, labels in train_dl:
        input_ids = tokens['input_ids'].to(device)
        attention_mask = tokens['attention_mask'].to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    print(f"Epoch {epoch+1}: Train Loss = {total_loss:.4f}, Accuracy = {correct/total:.4f}")


Epoch 1: Train Loss = 399.8131, Accuracy = 0.8557
Epoch 2: Train Loss = 227.8634, Accuracy = 0.9293
Epoch 3: Train Loss = 114.9483, Accuracy = 0.9676
Epoch 4: Train Loss = 60.3040, Accuracy = 0.9839
Epoch 5: Train Loss = 44.9911, Accuracy = 0.9879


In [50]:
model.eval()
val_correct, val_total = 0, 0
with torch.no_grad():
    for tokens, labels in val_dl:
        input_ids = tokens['input_ids'].to(device)
        attention_mask = tokens['attention_mask'].to(device)
        labels = labels.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)
        val_correct += (preds == labels).sum().item()
        val_total += labels.size(0)

val_acc = val_correct / val_total
print(f"Validation Accuracy = {val_acc:.4f}")


Validation Accuracy = 0.8655


In [65]:
def predict_sentiment(text, device):
    model.eval()
    tokens = bt(text, padding=True, truncation=True, return_tensors="pt", max_length=128)
    tokens = {k: v.to(device) for k, v in tokens.items()}

    with torch.no_grad():
        output = model(**tokens)
        logits = output.logits
        probs = torch.softmax(logits, dim=1)
        confidence, pred_class = torch.max(probs, dim=1)
        label = "Positive" if pred_class.item() == 1 else "Negative"
        print(f"Prediction: {label} ({confidence.item():.4f})")


string=str(input("Enter review:"))
predict_sentiment(string, device)

Enter review:it was a fine movie
Prediction: Positive (0.9985)
